In [98]:
import polars as pl
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np 

np.random.seed(42)

with open("jive.txt", "r") as f:
    data = [[value.strip() for value in line.split(',')] for line in f]

df = pl.DataFrame(data, schema=['yob', 'sob', 'qob', 'educ', 'lwage'], orient="row")

df = df.slice(1).with_columns(pl.all().cast(pl.Float64))

In [99]:
df = df.with_columns(
    (90 - pl.col("yob")).alias("age"),
    pl.col('lwage').exp().alias('wage')
)

print(df)

shape: (329_509, 7)
┌──────┬──────┬─────┬──────┬──────────┬──────┬────────────┐
│ yob  ┆ sob  ┆ qob ┆ educ ┆ lwage    ┆ age  ┆ wage       │
│ ---  ┆ ---  ┆ --- ┆ ---  ┆ ---      ┆ ---  ┆ ---        │
│ f64  ┆ f64  ┆ f64 ┆ f64  ┆ f64      ┆ f64  ┆ f64        │
╞══════╪══════╪═════╪══════╪══════════╪══════╪════════════╡
│ 30.0 ┆ 45.0 ┆ 1.0 ┆ 12.0 ┆ 5.790019 ┆ 60.0 ┆ 327.019238 │
│ 30.0 ┆ 45.0 ┆ 1.0 ┆ 11.0 ┆ 5.952494 ┆ 60.0 ┆ 384.711614 │
│ 30.0 ┆ 45.0 ┆ 1.0 ┆ 12.0 ┆ 5.315949 ┆ 60.0 ┆ 203.557598 │
│ 30.0 ┆ 45.0 ┆ 1.0 ┆ 12.0 ┆ 5.595926 ┆ 60.0 ┆ 269.326931 │
│ 30.0 ┆ 37.0 ┆ 1.0 ┆ 12.0 ┆ 6.068915 ┆ 60.0 ┆ 432.211478 │
│ …    ┆ …    ┆ …   ┆ …    ┆ …        ┆ …    ┆ …          │
│ 39.0 ┆ 9.0  ┆ 4.0 ┆ 16.0 ┆ 6.32398  ┆ 51.0 ┆ 557.788579 │
│ 39.0 ┆ 34.0 ┆ 4.0 ┆ 15.0 ┆ 5.847161 ┆ 51.0 ┆ 346.24998  │
│ 39.0 ┆ 34.0 ┆ 4.0 ┆ 12.0 ┆ 5.909597 ┆ 51.0 ┆ 368.557597 │
│ 39.0 ┆ 25.0 ┆ 4.0 ┆ 5.0  ┆ 6.047781 ┆ 51.0 ┆ 423.172967 │
│ 39.0 ┆ 36.0 ┆ 4.0 ┆ 19.0 ┆ 5.766817 ┆ 51.0 ┆ 319.519083 │
└──────┴──────┴─────

# 1 (a) Estimate simple model

In [100]:
Y = df['lwage'].to_pandas()
X = sm.add_constant(df['educ'].to_pandas())

model = sm.OLS(Y, X)
results = model.fit()

print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                 4.378e+04
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        11:47:00   Log-Likelihood:            -3.1935e+05
No. Observations:              329509   AIC:                         6.387e+05
Df Residuals:                  329507   BIC:                         6.387e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9952      0.004   1118.882      0.0

# 1 (b & c) Calculating standard errors 

In [101]:
standard_error = results.bse[1]
print(f"Standard Error for educ: {standard_error}")

clustered_se = results.get_robustcov_results(cov_type='cluster', groups=df['sob'].to_pandas()).bse
print(f"Clustered Standard Error for educ: {clustered_se}")

Standard Error for educ: 0.00033860679465786054
Clustered Standard Error for educ: [0.03065749 0.00176571]


C:\Users\cemkozan\AppData\Local\Temp\ipykernel_6608\1258157669.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  standard_error = results.bse[1]


In [102]:
# Manual calculation of OLS standard error

N = len(df)
residuals_i = results.resid.to_numpy()
X_i = X['educ'].to_numpy()
X_bar = X_i.mean()

homoskedastic_se = np.sqrt((1/N) * sum((residuals_i)**2) / (sum((X_i - X_bar)**2)))

print(homoskedastic_se)

0.0003386057670456429


In [103]:
# Manual calculation of clustered standard error

P = X.shape[1]
Omega = np.zeros((P, P))

n_clusters = df['sob'].to_pandas().nunique()
clusters = df['sob'].to_pandas().unique()


for cluster in clusters:
    mask = df['sob'].to_pandas() == cluster
    e_c = results.resid[mask].to_numpy()
    X_c = X[mask].to_numpy()

    score_c = X_c.T @ e_c
    
    cluster_matrix = np.outer(score_c, score_c)

    Omega += cluster_matrix


correction_factor = (N / (N - P)) * (n_clusters / (n_clusters - 1))

Omega *= correction_factor

XTX_inv = np.linalg.inv(X.T @ X)
clustered_se_manual = np.sqrt(np.diag(XTX_inv @ Omega @ XTX_inv))

print(f"Manually Calculated Clustered Standard Error for educ: {clustered_se_manual}")


Manually Calculated Clustered Standard Error for educ: [0.03065754 0.00176571]


# 1 (d) Bootstrapping

In [104]:
# Estimating OLS standard error by using the bayesian bootstrap:
B = 10000

# Take the residuals from the previous OLS regression
residuals = results.resid.to_numpy()

betas = []

for b in range(B):
    # Resample the residuals with replacement
    resampled_residuals = np.random.choice(residuals, size=N, replace=True)

    # Create the bootstrap sample
    X_boot = X.to_numpy()
    y_boot = X_boot @ results.params + resampled_residuals

    model_boot = sm.OLS(y_boot, X_boot)
    results_boot = model_boot.fit()
    beta_boot = results_boot.params[1]

    betas.append(beta_boot)

bootstrap_ols_se = np.sqrt((1/(B-1)) * sum((betas - np.mean(betas))**2))

print(f"Bootstrap OLS Standard Error for educ: {bootstrap_ols_se}")

Bootstrap OLS Standard Error for educ: 0.0003356398204111961


In [105]:
# Estimating clustered standard errors by bootstrapping:
B = 10000

betas = []

for b in range(B):
    # Resample the clusters themselves:
    sampled_clusters = np.random.choice(clusters, size=n_clusters, replace=True)

    sample_df = []

    for cluster in sampled_clusters:
        # Get observations in cluster, then resample them:

        cluster_data = df.filter(pl.col("sob") == cluster)
        resampled_cluster_data = cluster_data.sample(n=len(cluster_data), with_replacement=True)
        sample_df.append(resampled_cluster_data)

    # Recalculate the OLS estimate for this bootstrap sample
    sample_df = pl.concat(sample_df)
    X_boot = sm.add_constant(sample_df['educ'].to_numpy())
    y_boot = sample_df['lwage'].to_numpy()

    model_boot = sm.OLS(y_boot, X_boot)
    results_boot = model_boot.fit()
    beta_boot = results_boot.params[1]

    betas.append(beta_boot)

bootstrap_cluster_se = np.sqrt((1/(B-1)) * sum((betas - np.mean(betas))**2))

print(f"Bootstrap Clustered Standard Error for educ: {bootstrap_cluster_se}")

Bootstrap Clustered Standard Error for educ: 0.0017811296436398563


# 1 (e) Estimate Second model

In [106]:
# Add a new column to the dataframe with the average years of education for all individuals from the same state
df_with_state_avg = df.with_columns(
    df.group_by("sob").agg(pl.col("educ").mean().alias("state_avg_educ")).join(df.select("sob"), on="sob")["state_avg_educ"]
)

# Create X matrix with constant, education, and state average education
X_adj = sm.add_constant(df_with_state_avg.select(['educ', 'state_avg_educ']).to_pandas())

# Fit the model
model_adj = sm.OLS(Y, X_adj)
results_adj = model_adj.fit()

print(results_adj.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                 2.312e+04
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        12:12:28   Log-Likelihood:            -3.1827e+05
No. Observations:              329509   AIC:                         6.365e+05
Df Residuals:                  329506   BIC:                         6.366e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.1542      0.019    223.

# 1 (f & g) Calculate standard errors

In [107]:
# Calculate the errors using off-the-shelf package for sanity check
standard_error_adj = results_adj.bse
print(f"Standard Error for educ (adjusted): {standard_error_adj}")

clustered_se_adj = results_adj.get_robustcov_results(cov_type='cluster', groups=df_with_state_avg['sob'].to_pandas()).bse
print(f"Clustered Standard Error for educ (adjusted): {clustered_se_adj}")

Standard Error for educ (adjusted): const             0.018609
educ              0.000347
state_avg_educ    0.001495
dtype: float64
Clustered Standard Error for educ (adjusted): [0.1579015  0.00135076 0.0119502 ]


In [108]:
# Calculate the OLS standard error manually for the new model

N = len(df)
residuals_i = results_adj.resid.to_numpy()

X_educ = X_adj['educ'].to_numpy()
X_bar = X_educ.mean()

educ_homoskedastic_se = np.sqrt((1/N) * sum((residuals_i)**2) / (sum((X_educ - X_bar)**2)))

X_avgeduc = X_adj['state_avg_educ'].to_numpy()
X_bar = X_avgeduc.mean()

avgeduc_homoskedastic_se = np.sqrt((1/N) * sum((residuals_i)**2) / (sum((X_avgeduc - X_bar)**2)))

print(educ_homoskedastic_se)
print(avgeduc_homoskedastic_se)

0.0003374982597896972
0.0014546523790667121


In [109]:
# Calculate the clustered standard error manually for the new model

P = X_adj.shape[1]
Omega = np.zeros((P, P))

n_clusters = df['sob'].to_pandas().nunique()
clusters = df['sob'].to_pandas().unique()


for cluster in clusters:
    mask = df['sob'].to_pandas() == cluster
    e_c = results_adj.resid[mask].to_numpy()
    X_c = X_adj[mask].to_numpy()

    score_c = X_c.T @ e_c
    
    cluster_matrix = np.outer(score_c, score_c)

    Omega += cluster_matrix


correction_factor = (N / (N - P)) * (n_clusters / (n_clusters - 1))

Omega *= correction_factor

XTX_inv = np.linalg.inv(X_adj.T @ X_adj)
clustered_se_manual = np.sqrt(np.diag(XTX_inv @ Omega @ XTX_inv))

print(f"Manually Calculated Clustered Standard Error for educ: {clustered_se_manual}")


Manually Calculated Clustered Standard Error for educ: [0.15790174 0.00135076 0.01195022]


# 1 (h) Bootstrapping errors for new model

In [110]:
# Estimating OLS standard error by using the bayesian bootstrap:
B = 10000

# Take the residuals from the previous OLS regression
residuals = results_adj.resid.to_numpy()

betas = []

for b in range(B):
    # Resample the residuals with replacement
    resampled_residuals = np.random.choice(residuals, size=N, replace=True)

    # Create the bootstrap sample
    X_boot = X_adj.to_numpy()
    y_boot = X_boot @ results_adj.params + resampled_residuals

    model_boot = sm.OLS(y_boot, X_boot)
    results_boot = model_boot.fit()
    beta_boot = results_boot.params

    betas.append(beta_boot)

# Convert to numpy array for easier calculation
betas = np.array(betas)

# Calculate standard error for each coefficient
bootstrap_ols_se = np.sqrt((1/(B-1)) * np.sum((betas - np.mean(betas, axis=0))**2, axis=0))

print(f"Bootstrap OLS Standard Errors: {bootstrap_ols_se}")
print(f"  Intercept: {bootstrap_ols_se[0]}")
print(f"  Education: {bootstrap_ols_se[1]}")
print(f"  State Avg Education: {bootstrap_ols_se[2]}")

Bootstrap OLS Standard Errors: [0.01858613 0.0003446  0.00149249]
  Intercept: 0.01858613259405111
  Education: 0.0003445970650184384
  State Avg Education: 0.001492488201572065


In [111]:
# Estimating clustered standard errors by bootstrapping:
B = 10000

betas = []

for b in range(B):
    # Resample the clusters themselves:
    sampled_clusters = np.random.choice(clusters, size=n_clusters, replace=True)

    sample_df = []

    for cluster in sampled_clusters:
        # Get observations in cluster, then resample them:

        cluster_data = df_with_state_avg.filter(pl.col("sob") == cluster)
        resampled_cluster_data = cluster_data.sample(n=len(cluster_data), with_replacement=True)
        sample_df.append(resampled_cluster_data)

    # Recalculate the OLS estimate for this bootstrap sample
    sample_df = pl.concat(sample_df)
    X_boot = sm.add_constant(sample_df['educ', 'state_avg_educ'].to_numpy())
    y_boot = sample_df['lwage'].to_numpy()

    model_boot = sm.OLS(y_boot, X_boot)
    results_boot = model_boot.fit()
    beta_boot = results_boot.params

    betas.append(beta_boot)

# Calculate standard error for each coefficient
bootstrap_cluster_se = np.sqrt((1/(B-1)) * np.sum((betas - np.mean(betas, axis=0))**2, axis=0))

print(f"Bootstrap OLS Standard Errors: {bootstrap_cluster_se}")
print(f"  Intercept: {bootstrap_cluster_se[0]}")
print(f"  Education: {bootstrap_cluster_se[1]}")
print(f"  State Avg Education: {bootstrap_cluster_se[2]}")

Bootstrap OLS Standard Errors: [0.16124026 0.00137655 0.01222214]
  Intercept: 0.16124025970780442
  Education: 0.0013765547067013898
  State Avg Education: 0.012222135938806456
